#### 1、经典的代码实现过程.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# 定义硬件GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 原始数据加载
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# DataLoader定义
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# 模型定义
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)  # 输入1个通道，输出32个通道，卷积核5x5
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)  # 输入32个通道，输出64个通道
        self.fc1 = nn.Linear(64 * 4 * 4, 128)  # 64个通道，经过2次卷积后尺寸缩小为4x4
        self.fc2 = nn.Linear(128, 10)  # 输出10类

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(-1, 64 * 4 * 4)  # 展平
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)

# 损失函数与优化器定义
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 训练集循环
num_epochs = 5
train_losses = []

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 清零梯度
        optimizer.zero_grad()

        # 前向传播 + 反向传播 + 优化
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# 学习曲线绘制
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.legend()
plt.show()

# 模型保存与加载
torch.save(model.state_dict(), 'mnist_cnn.pth')
print("Model saved as mnist_cnn.pth")

# 模型推理
model.load_state_dict(torch.load('mnist_cnn.pth'))
model.eval()  # 设置模型为评估模式

# 测试推理
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


#### 2、定义一个config配置文件-指定相关参数

要将超参数如 `batch_size` 等设置写到一个配置文件中，可以使用多种格式，例如 YAML、JSON 或 INI 文件。这里，我们将使用 YAML 格式来定义一个配置文件，并展示如何在 PyTorch 中读取该配置文件。

### 第一步：创建 YAML 配置文件

创建一个名为 `config.yaml` 的文件并写入以下内容：

```yaml
# config.yaml
training:
  batch_size: 64
  num_epochs: 10
  learning_rate: 0.01
  momentum: 0.9

data:
  data_root: './data'
  num_workers: 4

model:
  input_channels: 1
  num_classes: 10
```

### 第二步：在 Python 中读取 YAML 配置文件

使用 `PyYAML` 库读取配置文件。在 Python 脚本中，首先确保安装 `PyYAML`：

```bash
pip install pyyaml
```

然后可以使用以下代码读取配置文件并访问超参数：

```python
import yaml

# 读取 YAML 配置文件
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# 访问参数
batch_size = config['training']['batch_size']
num_epochs = config['training']['num_epochs']
learning_rate = config['training']['learning_rate']
momentum = config['training']['momentum']
data_root = config['data']['data_root']
num_workers = config['data']['num_workers']
input_channels = config['model']['input_channels']
num_classes = config['model']['num_classes']

print(f'Batch Size: {batch_size}')
print(f'Num Epochs: {num_epochs}')
print(f'Learning Rate: {learning_rate}')
print(f'Momentum: {momentum}')
print(f'Data Root: {data_root}')
print(f'Num Workers: {num_workers}')
print(f'Input Channels: {input_channels}')
print(f'Num Classes: {num_classes}')
```

### 第三步：在 PyTorch 中使用超参数

在训练循环或模型定义中使用这些从配置文件中读取的超参数。例如：

```python
# 使用配置中的超参数创建数据加载器
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

# 使用这些超参数进行训练
for epoch in range(num_epochs):
    # 训练过程
```

### 总结
通过将超参数保存在配置文件中，可以更轻松地管理和调整这些参数，而不需要在代码中直接修改。这使得代码更加灵活和可维护。

In [ ]:
# 主程序部分.
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import yaml

# 读取 YAML 配置文件
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# 超参数
batch_size = config['training']['batch_size']
num_epochs = config['training']['num_epochs']
learning_rate = config['training']['learning_rate']
momentum = config['training']['momentum']
num_workers = config['data']['num_workers']

# 定义硬件GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 原始数据加载
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root=config['data']['data_root'], train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root=config['data']['data_root'], train=False, download=True, transform=transform)

# DataLoader定义
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# 模型定义
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(64 * 4 * 4, 128)  # 经过卷积和池化后的输出大小
        self.fc2 = nn.Linear(128, config['model']['num_classes'])

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(-1, 64 * 4 * 4)  # 展平
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)

# 损失函数与优化器定义
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

# 训练集循环
train_losses = []

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 清零梯度
        optimizer.zero_grad()

        # 前向传播 + 反向传播 + 优化
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# 学习曲线绘制
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.legend()
plt.show()

# 模型保存与加载
torch.save(model.state_dict(), 'mnist_cnn.pth')
print("Model saved as mnist_cnn.pth")

# 模型推理
model.load_state_dict(torch.load('mnist_cnn.pth'))
model.eval()  # 设置模型为评估模式

# 测试推理
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')
